Rapport : Quantitative Structure-Activity Relationship
Equipe : 

1 - Réprésentation des données

a)

b)

c)

2 - Mesures de distance

a)

b)

3 - Choix du modèle de classification

b)

4 - Application

In [ ]:
#okokoibn
def fun():
    return 2